In [1]:
import json
import os
import sys
from pprint import pprint

sys.path.append('..')

from utils.json_to_dict import MakeDict

In [2]:
# Make Nested JSON
with open('SampleJSONs' + os.sep + 'f2b_new_cats_vs_dogs_example.json') as f:
    inputs = json.load(f)
inputs = MakeDict(inputs).parse()

In [3]:
# Cell for all constants

TEST_DIR = ".." + os.sep + 'test' + os.sep
DATA_DIR = os.path.join(os.getcwd(), ".." + os.sep + 'data' + os.sep)
CATS_DIR = os.path.join(DATA_DIR, 'dogs_and_cats')

In [4]:
symbol_table = {
    'Conv2D' : 'model.add(Conv2D())',
    'MaxPooling2D' : 'model.add(MaxPooling2D())',
    'Dense': 'model.add(Dense())',
    'Flatten': 'model.add(Flatten())',
    'sgd': 'opt = SGD(lr=0.001, momentum=0.9)',
}

In [5]:
def get_imports():
    """
    imports the needed modules 
    """
    try:
        return \
        """
# File generated by DLMML Parser
import sys
import os
import numpy
import pandas
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
"""
    except Exception as e:
        print(e)

In [6]:
def init_sequential():
    """
    initializes the model
    """
    try:
        return \
'\nmodel = Sequential()\n'

    except Exception as e:
        print(e)

In [7]:
def image_input(inputs):
    """
    req in input json -
    dataset-path -> path to dataset
    image-augment -> params for image augmentation if required
    params -> params for building generator
    """
    base = inputs['dataset']['path']
    test_dir = os.path.join(base, 'test')
    train_dir = os.path.join(base, 'train')
    
    paths = \
"""

base = '{}'
train_dir = os.path.join(base, 'train')
test_dir = os.path.join(base, 'test')
""".format(inputs['dataset']['path'])

    generators = \
"""
augment = {}
kwargs = {}

train_datagen = ImageDataGenerator(**augment)
test_datagen = ImageDataGenerator(**augment)
train_generator = train_datagen.flow_from_directory(train_dir, **kwargs)
test_generator = test_datagen.flow_from_directory(test_dir, **kwargs)

""".format(inputs['image']['augment'], inputs['image']['params'])

    return paths+generators


In [8]:
def parse(input_dict):
    """
    Parser which adds layers (mostly)
    """
    try:
        # init_sequential()
        layers = input_dict['layers']
        generated_code = ''

        #TODO: Take care of indentation and stuff if making functions
        #TODO: Ordering in dict (work around -> ordered_dict)

        for layer in layers:
            name = layer.get('name', None)
            curr_layer = symbol_table[name]
            args = layer.copy()
            args.pop('name')
            args = str(args)
            curr_layer = curr_layer[:-2] + '**' + args + curr_layer[-2:]
            generated_code += curr_layer + '\n'
        print()
        return generated_code
    
    except Exception as e:
        print(e)


In [9]:
def compile_model(inputs):
    """
    Compiles model along with adding optimizer
    """
    try:
        opt = inputs.get("optimizer", "sgd")
        opt_code = symbol_table[opt]
        # let loss and metrics be a compulsory fields for the user?
        loss = inputs.get("loss")
        metrics = str(inputs.get("metrics"))

        return '\n' + opt_code + '\n' + \
"model.compile(optimizer=opt, loss='{}', metrics={})".format(loss, metrics)

    except Exception as e:
        print(e)

In [10]:
def add_plots_and_summary(fname="test"):
    return \
"""
def summarize_diagnostics(history, save_plots):
    # plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')

    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')

    # save plot to file
    if save_plots:
        filename = '{}'
        pyplot.savefig(filename + '_plot.png')
        pyplot.close()
""".format(fname)

In [11]:
def train_evaluate_model(inputs):
    """
    Adds code required to train and evaluate model
    Works only if:
        - train_data's generator is train_generator
        - test_data's generator is called test_generator
        - Only epochs and verbose taken from json
    """
    try:
            epochs = inputs.get('epochs', 20) #decide default
            verbose = inputs.get('verbose', 0)
            fit_generator = \
"""

print("\\n\\n ==========Fitting Model========== \\n")
history = model.fit_generator(
                    train_generator, 
                    steps_per_epoch=len(train_generator),
                    validation_data=test_generator, 
                    validation_steps=len(test_generator), 
                    epochs={}, 
                    verbose={}
                )
""".format(epochs, verbose)
            
            eval_generator = \
"""            

print("\\n\\n ==========Evalutating Model========== \\n")
_, acc = model.evaluate_generator(test_generator, steps=len(test_generator), verbose={})
print('\\n\\nACCURACY:  %.3f \\n\\n' % (acc * 100.0))
""".format(verbose)
            return fit_generator + eval_generator
    except Exception as e:
        print(e)


In [12]:
def plot_and_summarize_model(inputs):
    if eval(inputs.get('plot', False)):
        return \
"""

# Plotting graphs and Summarizing Model
summarize_diagnostics(history, {})
""".format(eval(inputs.get('save_plots', False)))

    else:
        return ''

In [13]:
def write_to_file(file_name, input_dict):
    """
    file_name: string (with extension), generated code \
               will be saved as file_name in the test dir
    input_dict: Dict generated by parsing json to make \
                nested dict
    """
    try:
        inputs = input_dict
        with open(TEST_DIR + file_name, 'w') as f:
            f.write(get_imports())
            f.write(add_plots_and_summary())
            f.write(image_input(inputs))
            f.write(init_sequential())
            f.write(parse(inputs))
            f.write(compile_model(inputs))
            f.write(train_evaluate_model(inputs))
            f.write(plot_and_summarize_model(inputs))
        print("Code generated in file test/" + file_name)
    except:
        print("Exception occured")

In [14]:
write_to_file('test.py', inputs)


Code generated in file test/test.py
